In [ ]:
import sys
import os
import re
import numpy as np
import pandas as pd
import time
import psycopg2
import requests
from bs4 import BeautifulSoup

from convertPDF.webScrape.pdf2xml import *
from convertPDF.webScrape.xml2txt import *

from convertPDF.driver import parseFullDay

from convertPDF.infoFns.genInfoFns import parseGenInfo
from convertPDF.infoFns.horseInfoFns import parseHorseInfo
from convertPDF.infoFns.timesInfoFns import parseTimeInfo
from convertPDF.infoFns.betInfoFns import parseBetInfo
from convertPDF.infoFns.runlineInfoFns import parseRunlineInfo
from convertPDF.infoFns.endInfoFns import parseEndInfo

from populateDB import generateEntries, populateDB

from convertPDF.infoFns.regexPatterns import *

from convertPDF.webScrape.htmlMgmt import *
from convertPDF.webScrape.saveTrackUrlsFromFiles import saveRaceUrlsFromFiles

from dataClean.cleanFns import racesClean, horsesClean

In [ ]:
horsesdf = pd.read_csv('./../outputs/horses.csv', dtype='string')
racesdf = pd.read_csv('./../outputs/races.csv', dtype='string')
horsesClean = horsesClean(horsesdf)
racesdf = racesClean(racesdf)

In [ ]:
start = time.time()
entries = generateEntries('./../charts/txts')
end = time.time()

end - start

In [ ]:
conn = psycopg2.connect(
    host = "localhost",
    database = "horses",
    user = "karisch",
    password = "cocacola",
    port = 5432
)

In [ ]:
populateDB(conn, entries)

In [ ]:
with conn.cursor() as cur:
    cur.execute("SELECT * FROM main.races;")
    races = cur.fetchall()
    cur.execute("SELECT * FROM main.horses;")
    horses = cur.fetchall()

In [ ]:
raceCols = ("track", "date","race","stakes","distanceStr","surface","weather","conditions",            "startTime","startNote","segment1","segment2","segment3","segment4","segment5","segments","fracTime1","fracTime2","fracTime3","fracTime4","fracTime5","finalTime","runup","wpsPool","firstPlaceWin","firstPlacePlace","firstPlaceShow","secondPlacePlace","secondPlaceShow","thirdPlaceShow","exactaBuyin","exactaFinish","exactaPayout","exactaPool","trifectaBuyin","trifectaFinish","trifectaPayout","trifectaPool")

In [ ]:
horseCols = ("track","date","race","horseProgram","horseName","lastRaceDay","lastRaceMonth","lastRaceYear","lastRaceTrack","jockey","weight","ME","placePP","placeSeg1","placeSeg2","placeSeg3","placeSeg4","placeSeg5","placeSeg6","odds","comments","lastRaceNum","lastRacePlace","lengthsSeg1","lengthsSeg2","lengthsSeg3","lengthsSeg4","lengthsSeg5","lengthsSeg6","rlLengthsSeg1","rlLengthsSeg2","rlLengthsSeg3","rlLengthsSeg4","rlLengthsSeg5","rlLengthsSeg6","rlPlaceSeg1","rlPlaceSeg2","rlPlaceSeg3","rlPlaceSeg4","rlPlaceSeg5","rlPlaceSeg6","trainer","owner")

In [ ]:
racesdf = pd.DataFrame(races, columns=raceCols, dtype='string')
horsesdf = pd.DataFrame(horses, columns=horseCols, dtype='string')

In [ ]:
racesdf.to_csv('./../outputs/races.csv', index=False)
horsesdf.to_csv('./../outputs/horses.csv', index=False)